In [ ]:
# Mount Google Drive to /content/gdive dir
from google.colab import drive
drive.mount('/content/gdrive')

# List the files in the My Drive dir
!ls "/content/gdrive/My Drive/"

In [ ]:
!mkdir COCO

!wget http://images.cocodataset.org/zips/train2017.zip
!unzip train2017.zip
!rm train2017.zip

!wget http://images.cocodataset.org/zips/val2017.zip
!unzip val2017.zip
!rm val2017.zip

#!wget http://images.cocodataset.org/zips/test2017.zip
#!unzip test2017.zip
#!rm test2017.zip


!cd ../
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip
!rm annotations_trainval2017.zip

In [ ]:
from pycocotools.coco import COCO
import os
import cv2
import time
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import random

In [ ]:
# Annotation Files
train_ann = "/content/annotations/instances_train2017.json"
val_ann = "/content/annotations/instances_val2017.json"

train_img_dir = '/content/train2017/'
val_img_dir = '/content/val2017/'

train_dest_segmentation_masks = '/content/segmentations/train'
val_dest_segmentation_masks = '/content/segmentations/val'

In [ ]:
# Initialize the COCO objects
train_coco = COCO(train_ann)
val_coco = COCO(val_ann)

loading annotations into memory...
Done (t=15.53s)
creating index...
index created!
loading annotations into memory...
Done (t=0.46s)
creating index...
index created!


In [ ]:
def get_mask(coco_object, index, filtered_cat_id, filtered_img_id, img_dir):
    # Load Image
    img_meta = coco_object.loadImgs(filtered_img_id[index])
    img_path = os.path.join(img_dir, img_meta[0]['file_name'])
    img = cv2.imread(img_path)

    # Load Annotations
    annIds = coco_object.getAnnIds(imgIds=img_meta[0]['id'], catIds=filtered_cat_id, iscrowd=None)
    anns = coco_object.loadAnns(annIds)
    
    # Generate Mask
    anns_img = np.zeros((img.shape[:-1]), dtype=np.uint8)
    for ann in anns:
        anns_img = np.maximum(anns_img, coco_object.annToMask(ann)*ann['category_id'])
        
    return img_meta[0]['file_name'], anns_img * 255


def write_mask(coco_object, index, filtered_cat_id, filtered_img_id, dest_folder, img_dir):
    if not os.path.isdir(dest_folder):
        os.makedirs(dest_folder)
        
    filename, mask = get_mask(coco_object, index, filtered_cat_id, filtered_img_id, img_dir)
    dest_path = os.path.join(dest_folder, filename)
    cv2.imwrite(dest_path, mask)


def write_train_masks(train_coco, train_dest_folder, classes=['person']):
    print("Processing Train Images")
    time.sleep(0.5)
    filtered_cat_id = train_coco.getCatIds(classes)
    filtered_img_id = train_coco.getImgIds(catIds=filtered_cat_id) 
    
    for i in tqdm(range(len(filtered_img_id))):
        write_mask(train_coco, i, filtered_cat_id, filtered_img_id, train_dest_folder, train_img_dir)
        
    print("Train Segmentation Masks Processing Complete")


def write_val_masks(val_coco, val_dest_folder, classes=['person']):
    print("Processing Val Images")
    time.sleep(0.5)
    filtered_cat_id = val_coco.getCatIds(classes)
    filtered_img_id = val_coco.getImgIds(catIds=filtered_cat_id) 
    
    for i in tqdm(range(len(filtered_img_id))):
        write_mask(val_coco, i, filtered_cat_id, filtered_img_id, val_dest_folder, val_img_dir)
        
    print("Val Segmentation Masks Processing Complete")


In [ ]:
write_train_masks(train_coco, train_dest_segmentation_masks)
write_val_masks(val_coco, val_dest_segmentation_masks)

Processing Train Images


100%|██████████| 64115/64115 [16:50<00:00, 63.45it/s]


Train Segmentation Masks Processing Complete
Processing Val Images


100%|██████████| 2693/2693 [00:34<00:00, 77.10it/s]

Val Segmentation Masks Processing Complete


In [ ]:
# Save results from segmentation
!zip -r "/content/gdrive/My Drive/segmentations.zip" /content/segmentations

In [ ]:
#!unzip "/content/gdrive/My Drive/segmentations.zip"
#!mv /content/content/segmentations/train /content/COCO/masks/
#!mv /content/content/segmentations/val /content/COCO/masks/